### Validação cruzada
Exemplo de validação cruzada em algoritimos de machine learning

In [48]:
import pandas as pd

uri = 'https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv'
data = pd.read_csv(uri)

new_columns_name = {
    'preco' : 'price',
    'idade_do_modelo' : 'model_age',
    'km_por_ano' : 'km_per_year',
    'vendido' : 'sold'
}

data = data.drop(columns=['Unnamed: 0'], axis=1)
data = data.rename(columns=new_columns_name)

data.head()

,price,sold,model_age,km_per_year
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [49]:
import numpy as np

from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

x = data[['price', 'model_age', 'km_per_year']] # X são os dados que queremos analisar
y = data['sold'] # Y são os dados que usaremos para classificar os dados de X

SEED = 158020 # Número aleatório usado pelo algoritimo de classificação (IMPORTANTE SETAR ELE PARA NÃO ALTERAR O VALOR FUTURAMENTE)
TEST_PERCENTAGE = 0.25 # Constante para indicar a porcentagem de dados que queremos testar (25%)

np.random.seed(SEED)

# train_x = Dados de treino de X
# test_x = Dados de teste de X
# train_y = Dados de treino de Y
# test_y = Dados de teste de Y

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=TEST_PERCENTAGE, stratify=y) # O parametro "stratify" 

print('We are going to train with %d elements and test with %d.' % (len(train_x), len(test_x)))

We are going to train with 7500 elements and test with 2500.


### Base-line
Definição de uma base-line para decidir uma taxa de acerto aceitavel

In [50]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy='stratified') # Instancia um classificador fictício
dummy_stratified.fit(train_x, train_y) # Treina o classificador com os dados de treino

dummy_accuracy = dummy_stratified.score(test_x, test_y) * 100 # Método que obtem a taxa de acerto do classificador fictício 

print('The accuracy of dummy stratified was %.2f%%' % dummy_accuracy)

The accuracy of dummy stratified was 50.96%


### Árvore de decisão
Algoritimo de classificação baseado na estrutura de árvore

In [51]:
from sklearn.tree import DecisionTreeClassifier

TREE_MAX_DEPTH = 2 # Número máx. de profundidade que o algoritimo deve legar em consideração para executar todos os dados
SEED = 158020 # Número aleatório usado pelo algoritimo de classificação (IMPORTANTE SETAR ELE PARA NÃO ALTERAR O VALOR FUTURAMENTE)

np.random.seed(SEED)

model = DecisionTreeClassifier(max_depth=TREE_MAX_DEPTH) # Instancia um classificador de árvore
model.fit(train_x, train_y) # Treina o classificador com os dados de treino

predictions = model.predict(test_x) # Retorna as previsões de acerto que o classificador foi treinado com base nos dados de teste
model_accuracy = accuracy_score(test_y, predictions) * 100 # Método que obtem a taxa de acerto do test com base nas previsões de acerto

print('The model accuracy was %.2f%%' % model_accuracy)

The model accuracy was 71.92%


### Validação cruzada na prática

In [52]:
def print_model_results(results):
    average = results['test_score'].mean() # Média dos resultados do teste cruzado
    average_default_detour = results['test_score'].std() # Desvio padrão da média do teste cruzado

    print('Accuracy average: %.2f' % (average * 100))
    print('Accuracy detour: [%.2f, %.2f]' % ((average - 2 * average_default_detour) * 100, (average + 2 * average_default_detour) * 100))

In [53]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier

import numpy as np

CROSS_VALIDATION_NUMBER = 5 # Número de vezes que o algoritimo vai quebrar a amostra de dados para validar (5 É O PADRÃO UTILIZADO NO MERCADO)
TREE_MAX_DEPTH = 2 # Número máx. de profundidade que o algoritimo deve legar em consideração para executar todos os dados

# np.random.seed(SEED) # Para o cross validation não é mais preciso setar um número aleatório para o algoritomo usar

model = DecisionTreeClassifier(max_depth=TREE_MAX_DEPTH)
results = cross_validate(model, x, y, cv=CROSS_VALIDATION_NUMBER)

print_model_results(results)

Accuracy average: 75.78
Accuracy detour: [75.21, 76.35]


### Aleatóriedade no cross validate

In [54]:
from sklearn.model_selection import cross_validate, KFold
from sklearn.tree import DecisionTreeClassifier

TREE_MAX_DEPTH = 2 # Número máx. de profundidade que o algoritimo deve legar em consideração para executar todos os dados

cross_validation_number = KFold(n_splits=10) # Número de vezes que o algoritimo vai quebrar a amostra de dados para validar (5 É O PADRÃO UTILIZADO NO MERCADO)

model = DecisionTreeClassifier(max_depth=TREE_MAX_DEPTH)
results = cross_validate(model, x, y, cv=cross_validation_number)

print_model_results(results)

Accuracy average: 75.78
Accuracy detour: [74.37, 77.19]


In [55]:
TREE_MAX_DEPTH = 2 # Número máx. de profundidade que o algoritimo deve legar em consideração para executar todos os dados

# O parametro "shuffle" quebra a amostra de dados para gerar o número de dados para validar
cross_validation_number = KFold(n_splits=10, shuffle=True) # Número de vezes que o algoritimo vai quebrar a amostra de dados para validar (5 É O PADRÃO UTILIZADO NO MERCADO)

model = DecisionTreeClassifier(max_depth=TREE_MAX_DEPTH)
results = cross_validate(model, x, y, cv=cross_validation_number)

print_model_results(results)

Accuracy average: 75.78
Accuracy detour: [73.28, 78.28]


### Simulação horrivel de azar

Pode ser tanto um "azar" quanto um desbalanceamento na proporção de exemplos entre as classes.

In [56]:
unlucky_data = data.sort_values('sold', ascending=True) # Ordena os dados de forma crescente

unlucky_x = unlucky_data[['price', 'model_age', 'km_per_year']] # X são os dados que queremos analisar
unlucky_y = unlucky_data['sold'] # Y são os dados que usaremos para classificar os dados de X

unlucky_data.head()

,price,sold,model_age,km_per_year
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [57]:
from sklearn.model_selection import cross_validate, KFold
from sklearn.tree import DecisionTreeClassifier

TREE_MAX_DEPTH = 2 # Número máx. de profundidade que o algoritimo deve legar em consideração para executar todos os dados

cross_validation_number = KFold(n_splits=10) # Número de vezes que o algoritimo vai quebrar a amostra de dados para validar (5 É O PADRÃO UTILIZADO NO MERCADO)

model = DecisionTreeClassifier(max_depth=TREE_MAX_DEPTH)
results = cross_validate(model, unlucky_x, unlucky_y, cv=cross_validation_number)

print_model_results(results)

Accuracy average: 57.84
Accuracy detour: [34.29, 81.39]


In [58]:
from sklearn.model_selection import cross_validate, KFold
from sklearn.tree import DecisionTreeClassifier

TREE_MAX_DEPTH = 2 # Número máx. de profundidade que o algoritimo deve legar em consideração para executar todos os dados

cross_validation_number = KFold(n_splits=10, shuffle=True) # Número de vezes que o algoritimo vai quebrar a amostra de dados para validar (5 É O PADRÃO UTILIZADO NO MERCADO)

model = DecisionTreeClassifier(max_depth=TREE_MAX_DEPTH)
results = cross_validate(model, unlucky_x, unlucky_y, cv=cross_validation_number)

print_model_results(results)

Accuracy average: 75.78
Accuracy detour: [73.15, 78.41]


In [59]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

TREE_MAX_DEPTH = 2 # Número máx. de profundidade que o algoritimo deve legar em consideração para executar todos os dados

cross_validation_number = StratifiedKFold(n_splits=10, shuffle=True) # Número de vezes que o algoritimo vai quebrar a amostra de dados para validar (5 É O PADRÃO UTILIZADO NO MERCADO)

model = DecisionTreeClassifier(max_depth=TREE_MAX_DEPTH)
results = cross_validate(model, unlucky_x, unlucky_y, cv=cross_validation_number)

print_model_results(results)

Accuracy average: 75.78
Accuracy detour: [73.82, 77.74]


### Gerando uma coluna aleatória para testes

In [60]:
np.random.seed(SEED)

data_length = len(data)
data['random_model'] = data['model_age'] + np.random.randint(-2, 3, size=data_length) # Soma a idade do modelo com um número aleatório entre -2 e 2
data['random_model'] = data['random_model'] + abs(data['random_model'].min()) + 1 # Formata o menor número para um número absoluto e soma no modelo aleatório gerado

data.head()

,price,sold,model_age,km_per_year,random_model
0,30941.02,1,18,35085.22134,19
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,4


E classificando por grupo

In [66]:
from sklearn.model_selection import cross_validate, GroupKFold
from sklearn.tree import DecisionTreeClassifier

TREE_MAX_DEPTH = 2 # Número máx. de profundidade que o algoritimo deve legar em consideração para executar todos os dados

cross_validation_number = GroupKFold(n_splits=10) # Número de vezes que o algoritimo vai quebrar a amostra de dados para validar separado por grupos

model = DecisionTreeClassifier(max_depth=TREE_MAX_DEPTH)
results = cross_validate(model, unlucky_x, unlucky_y, groups=data['random_model'], cv=cross_validation_number)

print_model_results(results)

Accuracy average: 75.76
Accuracy detour: [72.91, 78.60]
